In [ ]:
!pip install pdfminer.six python-docx pytesseract opencv-python transformers nltk langdetect --quiet
!pip install rake_nltk
!pip install python-docx
!pip install pytesseract opencv-python
!pip install streamlit

In [1]:
import os
import pytesseract
import cv2
import nltk
import json
from pdfminer.high_level import extract_text
from docx import Document
from langdetect import detect
from transformers import pipeline
from langdetect import detect
from sklearn.feature_extraction.text import TfidfVectorizer
from datetime import datetime
from transformers import pipeline
from docx import Document

nltk.download('punkt')
nltk.download('stopwords')
print("Setup done ")

Setup done 


[nltk_data] Downloading package punkt to /Users/pirachi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pirachi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Extracting Text
pdf_path = "../sample_docs/sample.pdf"

# def extract_text_from_pdf(path):
#     try:
#         text = extract_text(path)
#         return text.strip()
#     except Exception as e:
#         return f"Error extracting text: {e}"

# # Run extraction
# pdf_text = extract_text_from_pdf(pdf_path)
# print("Text extracted. Preview:\n")
# print(pdf_text[:1000])

# def split_into_chunks(text, max_chars=1000):
#     """
#     Splits the full text into semi-intelligent chunks for summarization.
#     Breaks on paragraphs when possible.
#     """
#     paragraphs = text.split('\n')
#     chunks, current_chunk = [], ''

#     for para in paragraphs:
#         if len(current_chunk) + len(para) < max_chars:
#             current_chunk += para + '\n'
#         else:
#             chunks.append(current_chunk.strip())
#             current_chunk = para + '\n'
#     if current_chunk:
#         chunks.append(current_chunk.strip())

#     return chunks


def split_into_chunks(text, max_chars=1000):
    paragraphs = text.split('\n')
    chunks, current_chunk = [], ''
    for para in paragraphs:
        if len(current_chunk) + len(para) < max_chars:
            current_chunk += para + '\n'
        else:
            chunks.append(current_chunk.strip())
            current_chunk = para + '\n'
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

In [ ]:
# summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# # Keep first 1000 tokens or so
# chunk = pdf_text[:1000]

# summary = summarizer(chunk, max_length=100, min_length=30, do_sample=False)[0]['summary_text']
# print("Summary:\n", summary)

# def summarize_text(text, max_chars_per_chunk=1000):
#     chunks = split_into_chunks(text, max_chars=max_chars_per_chunk)
#     summaries = []
#     for i, chunk in enumerate(chunks):
#         try:
#             result = summarizer(chunk, max_length=100, min_length=30, do_sample=False)[0]['summary_text']
#             summaries.append(result)
#         except Exception as e:
#             print(f"Skipping chunk {i} due to error: {e}")
#             continue
#     return "\n".join(summaries)

def summarize_text_limited(text, summarizer, max_chars=1000, max_chunks=3):
    chunks = split_into_chunks(text, max_chars)
    summaries = []

    for i, chunk in enumerate(chunks[:max_chunks]):
        try:
            summary = summarize_text_limited(text, summarizer, max_chars=1000, max_chunks=3)
            summaries.append(summary)
        except Exception as e:
            print(f"Error summarizing chunk {i}: {e}")
            continue

    return "\n".join(summaries)

In [ ]:
def generate_metadata(path):
    print(f"\n📄 Processing: {path}")
    text = extract_text_auto(path)

    if not text or len(text.strip()) < 50:
        return {"error": "Insufficient content extracted."}

    # Load model once
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

    lang = detect(text)
    summary = summarize_text_limited(text, summarizer)
    keywords = extract_keywords(text, top_n=10)

    metadata = {
        "filename": os.path.basename(path),
        "extracted_on": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "language": lang,
        "summary": summary,
        "keywords": keywords.tolist() if hasattr(keywords, 'tolist') else list(keywords),
        "num_characters": len(text),
        "num_words": len(text.split())
    }

    # Save as JSON
    out_path = path + ".metadata.json"
    with open(out_path, "w") as f:
        json.dump(metadata, f, indent=4)

    print(f"✅ Metadata saved to: {out_path}")
    return metadata

In [ ]:
# Detecting Language
language = detect(pdf_text)
print(f"Detected Language: {language}")

In [ ]:


def extract_keywords(text, top_n=10):
    vectorizer = TfidfVectorizer(stop_words='english', max_features=top_n)
    X = vectorizer.fit_transform([text])
    return vectorizer.get_feature_names_out()

keywords = extract_keywords(pdf_text)
print("🔑 Keywords:")
print(keywords)

In [ ]:


metadata = {
    "filename": pdf_path.split("/")[-1],
    "extracted_on": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "language": language,
    "summary": summary,
    "keywords": keywords.tolist() if hasattr(keywords, 'tolist') else list(keywords),
    "num_characters": len(pdf_text),
    "num_words": len(pdf_text.split())
}

# Save metadata
with open("../sample_docs/sample_metadata.json", "w") as f:
    json.dump(metadata, f, indent=4)

print("✅ Metadata saved as 'sample_metadata.json'")

# DOC extrac

In [ ]:


def extract_text_from_docx(path):
    try:
        doc = Document(path)
        full_text = [para.text for para in doc.paragraphs]
        return "\n".join(full_text)
    except Exception as e:
        return f"Error extracting DOCX: {e}"

# TXT extraction

In [ ]:
def extract_text_from_txt(path):
    try:
        with open(path, "r", encoding="utf-8") as f:
            return f.read()
    except Exception as e:
        return f"Error extracting TXT: {e}"

# OCR for scanned images

In [ ]:


def extract_text_from_image(path):
    try:
        image = cv2.imread(path)
        text = pytesseract.image_to_string(image)
        return text.strip()
    except Exception as e:
        return f"Error in OCR: {e}"

## Detect file type and extraction

In [ ]:
def extract_text_auto(path):
    ext = path.split('.')[-1].lower()
    if ext == 'pdf':
        return extract_text_from_pdf(path)
    elif ext == 'docx':
        return extract_text_from_docx(path)
    elif ext == 'txt':
        return extract_text_from_txt(path)
    elif ext in ['jpg', 'jpeg', 'png']:
        return extract_text_from_image(path)
    else:
        return "Unsupported file format"

In [ ]:
path = "../sample_docs/sample.txt"  # or sample.docx, sample.png, etc.
text = extract_text_auto(path)
print(text[:1000])

In [ ]:
def generate_metadata(path):
    # Extract text from any file
    text = extract_text_auto(path)

    if not text or len(text.strip()) < 50:
        return {"error": "Insufficient text extracted from document."}

    # Language detection
    lang = detect(text)

    # Summarization (limit input length)
    chunk = text[:1000]
    summary = summarizer(chunk, max_length=100, min_length=30, do_sample=False)[0]['summary_text']

    # Keywords using TF-IDF
    kw = extract_keywords(text, top_n=10)

    # Final metadata
    metadata = {
        "filename": path.split("/")[-1],
        "extracted_on": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "language": lang,
        "summary": summary,
        "keywords": kw.tolist() if hasattr(kw, 'tolist') else list(kw),
        "num_characters": len(text),
        "num_words": len(text.split())
    }

    # Save metadata to JSON
    output_path = path.replace(".pdf", ".json").replace(".docx", ".json").replace(".txt", ".json").replace(".png", ".json").replace(".jpg", ".json")
    with open(output_path, "w") as f:
        json.dump(metadata, f, indent=4)

    print(f"✅ Metadata generated and saved to: {output_path}")
    return metadata


In [ ]:
generate_metadata("../sample_docs/sample.pdf")
generate_metadata("../sample_docs/sample.txt")
generate_metadata("../sample_docs/sample.docx")
# generate_metadata("../sample_docs/scan.jpg")
